In [ ]:
#functions needed for building a black jack game
#set up game:
#deal card
#check if gone over limit
#reset game
#point counting
#winner declaring
#score keeping
#declare a class for cards
#print results(doesn't have to be picture of a card, numbers should suffice)
#classes: Player, CardDeck, Dealer, Card

In [21]:
#import random for dealing cards
import random

gameInProgress = True
restartGame=True
bust = False
action = {'stand': True, 'hit':False}
quitGame=True
insuranceBet = 0
playerHand = []
dealerHand = []
placeInsurance = False
dealer = Dealer(dealerHand)
player = Player(playerHand)
deck = Deck()

In [22]:
class Player(object):
    
    def __init__(self,name = 'player1',bank=100, hand =[], bet = 0):
        self.name = name
        self.bank = bank
        self.hand = hand
        self.bet = bet
    
    def setName(self, playerName):
        self.name = playerName
    
    def addCard(self, card):
        self.hand.append(card)
        
    def placeBet(self):
        while True:
            try:
                bet = int(raw_input('How much would you like to bet?'))
            except:
                print 'Please Enter a valid bet.'
            else:
                if bet > self.bank:
                    print "You don't have enough money. Please enter valid bet."
                else:
                    self.bank -= bet
                    self.bet = bet
                    break
                    
    def clearBet(self):
        self.bet = 0
        
    def collectWins(self, winnings):
        self.bank += winnings
        
    def action(self, action):
        return action == True
    def checkHand(self):
        if len(self.hand)>0:
            for card in hand:
                print str(card.number) + ' '+ card.suit

In [23]:
class Card(object):
    def __init__ (self, hierarchy,suit, number):
        self.hierarchy = hierarchy
        self.suit = suit
        self.number = number
        

In [24]:
class Deck(object):
    def __init__ (self, deck=[], removedDeck=[]):
        self.deck = deck
        self.removedDeck = removedDeck
        
    def initDeck(self):
        del self.deck[:]
        del self.removedDeck[:]
        
        for x in xrange(1,5):
            for num in xrange(1,14):
                if x == 1:
                    card = Card(x,'Cloves',num)                    
                elif x == 2:
                    card = Card(x,'Diamonds',num)
                elif x == 3:
                    card = Card(x,'Hearts', num)
                elif x == 4:
                    card = Card(x,'Spades', num)
                self.deck.append(card)
                
    def shuffleDeck(self):
        random.shuffle(self.deck)
    
    def removeCard(self):
        self.checkForReShuffling()
        return self.deck.pop(random.randint(0,len(self.deck)))
    
    def recordDiscardedCard(self, player):
        for card in player.hand:
            self.removedDeck.append(card)
        del player.hand[:]
        
    def __len__(self):
        return len(self.deck)
    
    def checkForReShuffling(self):
        if len(self.deck)<=21:
            print 'Current deck contains less than 20 cards, \nremoved cards will be added back to deck and deck will be reshuffled'
            for card in self.removedDeck:
                self.deck.append(self.removedDeck.pop())
                self.shuffleDeck()

In [25]:
class Dealer(Player):
    def __init__(self,hand = []):
        Player.__init__(self, 'Dealer')
        self.hand = hand
    def dealCard(self, deck):
        return deck.removeCard()
    
    def shuffleDeck(self, deck):
        deck.shuffleDeck()

In [26]:
def checkPoints(player):
    total = 0
    containsAce = False
    bust = False
    for card in player.hand:
        if card.number>10:
            total+=10
        elif  card.number == 1:
            containsAce = True
            total+=10
        else:
            total+= card.number   
    if containsAce:
        if type(player) is Player:
            print player.name+"'s points (Ace as 10): "+ str(total)
            print player.name+"'s points (Ace as 1): "+ str(total-9)
            
        if total>21 & (total-9)>21:
            bust = True
        if total >21 & (total-9) < 21:
            total = (total-9)
    else:
        print player.name+"'s points: "+ str(total)
        if total>21:
            bust = True
            
    if (type(player) is Dealer) & containsAce:
        if total >21:
            total -= 9
            
    return {'points':total,'containsAce':containsAce, 'bust': bust}

In [27]:
def dealerAction(dealer,deck):
    if checkForBlackJack(dealer):
        return
    while True:        
        dealerPoints = checkPoints(dealer)
        if dealerPoints['bust'] or dealerPoints['points']>=17:
            print 'The dealer had busted!'
            break
        else:
            print "Dealer's current points: "+ str(dealerPoints['points'])
            dealer.addCard(dealer.dealCard(deck))

In [28]:
def playerAction(player, deck, dealer):
    acceptInsurance = False
    
    if player.bet > 0:
        player.placeBet()        
    
    if checkForBlackJack(player):
        return
    
    print "Dealer's first card: "+ dealer.hand[0].suit+' '+ str(dealer.hand[0].number)
    
    if dealer.hand[0].number == 1:
        print "dealer's first card is Ace!"
        askForInsurance(player)
        if checkForBlackJack(dealer):
            return        
        
    while True:
        playerPoints = checkPoints(player)
        if playerPoints['bust']:
            break
        elif playerPoints['containsAce']:
            if (playerPoints['points']-9)>21:
                break
            else:
                choice = askForInput()
                if choice.upper() == 'H':
                    player.addCard(dealer.dealCard(deck))
                else:
                    break
        else:
            if playerPoints['points']> 21:
                break
            else:
                choice = askForInput()
                if choice.upper() == 'H':
                    player.addCard(dealer.dealCard(deck))
                else:
                    break                

In [29]:
def askForInsurance(player):
    global insuranceBet,placeInsurance
    while True:
        choice = raw_input("Enter 'Y' to place insurance or 'N' to pass")
        if choice.upper() == 'Y' or choice.upper() == 'N':
            placeInsurance = True
            break
        else:
            print 'Please enter a valid input.'
    if placeInsurance:
        while True:
            try:
                insuranceBet = int(raw_input("Please place a bet: "))
            except:
                print 'Please place a valid bet.'
            else:
                if insuranceBet > player.bank:
                    print 'You have insufficient funds'
                elif insuranceBet > (player.bet*0.5):
                    print 'You cannot bet higher than 1/2 of your original bet.'
                else:
                    break
    return placeInsurance

In [30]:
def askForInput():
    while True:
        choice = raw_input("Enter 'H' to Hit or 'S' to Stand")
        if choice.upper() == 'H' or choice.upper() == 'S':
            break
        else: print 'Please enter a valid input.'
    return choice

In [31]:
def checkForBlackJack(player):
    blackJack = False
    if len(player.hand)<3:
        playerPoint = checkPoints(player)
        if playerPoint['points'] == 21:
            blackJack = True
    return blackJack

In [32]:
def declareWinner(player, dealer):
    global placeInsurance, insuranceBet, deck
    if checkForBlackJack(player) & checkForBlackJack(dealer):
        print 'Draw Game!'
        player.collectWins(player.bet)
        
    elif checkForBlackJack(player):
        print 'Black Jack for ' + player.name+'!'
        player.collectWins(player.bet*2.5)
        
    elif checkForBlackJack(dealer):
        print 'Black Jack for Dealer!'
        if placeInsurance:
            player.collectWins(insuranceBet*2+insuranceBet)
            
    else:
        playerPoint = checkPoints(player)
        dealerPoint = checkPoints(dealer)
        
        if playerPoint['bust']:
            print player.name +' had busted, Dealer Wins!'
            
        elif dealerPoint['bust']:
            print player.name +' wins!'
            player.collectWins(player.bet*2)
            
        elif playerPoint['points']> dealerPoint['points']:
            print player.name + ' wins!'
            player.collectWins(player.bet*2)
            
        elif playerPoint['points']< dealerPoint['points']:
            print 'Dealer wins!'
    deck.recordDiscardedCard(player)
    deck.recordDiscardedCard(dealer)

In [33]:
def showPlayerStats():
    global player
    print 'Your bank: '+ str(player.bank)

In [34]:
def newRound(player, dealer, deck):
    deck.recordDiscardedCard(player)
    deck.recordDiscardedCard(dealer)
    deck.checkForReShuffling()
    player.bet = 0
    insuranceBet = 0
    placeInsurance = False
    playGame()

In [35]:
def askForPlayerName():
    while True:
        name = raw_input('Please enter your name: ')
        if len(name.replace(' ',''))>0:
            return name
        else:
            print 'Please enter a valid name'

In [36]:
def dealCard(dealer, player, deck):
    player.addCard(dealer.dealCard(deck))    

In [37]:
def initGame():
    global player, dealer, deck
    print "Welcome to Fk's Black Jack Game!"
    choice = 'O'
    while True:
        choice = raw_input("Enter 'Y' to play or 'N' to leave")
        if choice.upper() != 'Y' and choice.upper() != 'N':
            print 'Please enter a valid input.'
        else:
            break
    if choice.upper() == 'N':
        print 'Good Bye!'
        return False
    else:
        deck.initDeck()
        dealer.shuffleDeck(deck)
        player.setName(askForPlayerName())
        playGame()       
        return True

In [38]:
def playGame():   
    global player, dealer, deck
    for x in range (1,3):
        dealCard(dealer, player, deck)
        dealCard(dealer, dealer, deck)
    playerAction(player, deck, dealer)
    dealerAction(dealer, deck)
    declareWinner(player, dealer)
    showPlayerStats()
    if continueGame():
        newRound(player, dealer, deck)
        playGame()
    else:
        print 'Good Bye!'
    

In [39]:
def continueGame():
    choice = raw_input("Enter 'Y' if you wish to continue to play, or any key to quit: ")
    if choice.upper() == 'Y':
        return True
    else:
        return False

In [40]:
def game():
    if initGame():
        playGame()

In [42]:
game()

Welcome to Fk's Black Jack Game!
Enter 'Y' to play or 'N' to leavey
Please enter your name: Fk


IndexError: pop index out of range

In [43]:
len(deck)

50

In [45]:
deck.deck.pop(random.randint(0,len(deck.deck)))